# **Chapter 7: Using Examples (Few-Shot Prompting)**

---
**Lesson:**

Giving Claude examples of how you want it to behave (or how you want it not to behave) is extremely effective for:
- Getting the right answer
- Getting the answer in the right format

This sort of prompting is also called "few shot prompting".

---

First we will setup our dependencies

In [2]:
%%capture
#Cell [1]

#langchain
%pip install langchain

#Install libraries 
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

#Import libraries
import json
import os
import sys
import boto3
import botocore

#Set up Bedrock runtime client
bedrock_runtime = boto3.client('bedrock-runtime')
boto3_bedrock = boto3.client('bedrock')

#Set up ModelID (Claude 3 Sonnet in this case)
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"

In [3]:
#Cell [2]

#Setup Model invocation function 
#Code modified from documentation: https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-runtime_example_bedrock-runtime_Claude3_Text_section.html
def get_response(prompt_data):
    response = bedrock_runtime.invoke_model(
                    modelId=modelId,
                    body=json.dumps(
                        {
                            "anthropic_version": "bedrock-2023-05-31",
                            "max_tokens": 1024,
                            "messages": [
                                {
                                    "role": "user",
                                    "content": [{"type": "text", "text": prompt_data}],
                                }
                            ],
                        }
                    ),
                )
    result = json.loads(response.get("body").read())
    response = result.get("content", [])[0]['text']
    return response

# **Examples:**

**Example 7.1 - Is Santa real?**

Pretend you're a developer trying to build a "parent bot" that responds to questions from kids. Claude's default response is quite formal and robotic. This is going to break a child's heart.

In [4]:
response = get_response("""

Human: Will Santa bring me presents on Christmas?

Assistant:""")

print(response)

I don't actually have definitive knowledge about whether Santa will bring you presents or not. Santa Claus is a legendary figure associated with the tradition of gift-giving during the Christmas holiday season. Many families enjoy pretending that Santa brings gifts to good children on Christmas Eve as part of the festive celebration and childhood wonder. However, the decision of what traditions and stories to share with children is up to their parents or guardians. If you believe in Santa, I certainly wouldn't want to disillusion you about the magic of the holidays. But you may also want to check with your parents about their approach to the Santa Claus tradition.


**Example 7.2 - Santa is real!**

To improve Claude's response, let's allow Claude to think things out first before answering. We do that by literally spelling out the steps that Claude should take in order to process and think through the prompt. 
Now, Claude's response is much better and more nuanced.

In [5]:
response = get_response("""

Human: Please complete the conversation by writing the next line, speaking as ""A"".
Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. There might be something waiting for you in the morning.
Q: Will Santa bring me presents on Christmas?

Assistant:""")

print(response)

A: Yes, absolutely! Santa will definitely bring you presents if you've been a good kid this year. Just make sure to leave out some cookies and milk for him on Christmas Eve.


**Example 7.3 - Extraction**

In the following formatting example, we could walk Claude step by step through a set of formatting instructions on how to extract names and professions and then format them exactly the way we want, or we could just provide Claude with some correctly-formatted examples and Claude can extrapolate from there.

In [6]:
response = get_response("""

Human: In the bustling town of Emerald Hills, a diverse group of individuals made their mark. Sarah Martinez, a dedicated nurse, was known for her compassionate care at the local hospital. David Thompson, an innovative software engineer, worked tirelessly on groundbreaking projects that would revolutionize the tech industry. Meanwhile, Emily Nakamura, a talented artist and muralist, painted vibrant and thought-provoking pieces that adorned the walls of buildings and galleries alike. Lastly, Michael O'Connell, an ambitious entrepreneur, opened a unique, eco-friendly cafe that quickly became the town's favorite meeting spot. Each of these individuals contributed to the rich tapestry of the Emerald Hills community.
<individuals>
1. Sarah Martinez [NURSE]
2. David Thompson [SOFTWARE ENGINEER]
3. Emily Nakamura [ARTIST]
4. Michael O'Connell [ENTREPRENEUR]
</individuals>

At the heart of the town, Chef Oliver Hamilton has transformed the culinary scene with his farm-to-table restaurant, Green Plate. Oliver's dedication to sourcing local, organic ingredients has earned the establishment rave reviews from food critics and locals alike.

Just down the street, you'll find the Riverside Grove Library, where head librarian Elizabeth Chen has worked diligently to create a welcoming and inclusive space for all. Her efforts to expand the library's offerings and establish reading programs for children have had a significant impact on the town's literacy rates.

As you stroll through the charming town square, you'll be captivated by the beautiful murals adorning the walls. These masterpieces are the work of renowned artist, Isabella Torres, whose talent for capturing the essence of Riverside Grove has brought the town to life.

Riverside Grove's athletic achievements are also worth noting, thanks to former Olympic swimmer-turned-coach, Marcus Jenkins. Marcus has used his experience and passion to train the town's youth, leading the Riverside Grove Swim Team to several regional championships.
<individuals>
1. Oliver Hamilton [CHEF]
2. Elizabeth Chen [LIBRARIAN]
3. Isabella Torres [ARTIST]
4. Marcus Jenkins [COACH]
</individuals>

Oak Valley, a charming small town, is home to a remarkable trio of individuals whose skills and dedication have left a lasting impact on the community.

At the town's bustling farmer's market, you'll find Laura Simmons, a passionate organic farmer known for her delicious and sustainably grown produce. Her dedication to promoting healthy eating has inspired the town to embrace a more eco-conscious lifestyle.

In Oak Valley's community center, Kevin Alvarez, a skilled dance instructor, has brought the joy of movement to people of all ages. His inclusive dance classes have fostered a sense of unity and self-expression among residents, enriching the local arts scene.

Lastly, Rachel O'Connor, a tireless volunteer, dedicates her time to various charitable initiatives. Her commitment to improving the lives of others has been instrumental in creating a strong sense of community within Oak Valley.

Through their unique talents and unwavering dedication, Laura, Kevin, and Rachel have woven themselves into the fabric of Oak Valley, helping to create a vibrant and thriving small town.

Assistant: <individuals>""")

print(response)

Here are the individuals mentioned in the passages, organized by town:

Emerald Hills:
1. Sarah Martinez [NURSE]
2. David Thompson [SOFTWARE ENGINEER] 
3. Emily Nakamura [ARTIST]
4. Michael O'Connell [ENTREPRENEUR]

Riverside Grove:
1. Oliver Hamilton [CHEF]
2. Elizabeth Chen [LIBRARIAN]
3. Isabella Torres [ARTIST]
4. Marcus Jenkins [COACH]

Oak Valley:
1. Laura Simmons [FARMER]
2. Kevin Alvarez [DANCE INSTRUCTOR]
3. Rachel O'Connor [VOLUNTEER]


**Example 7.4 - Copycat**

For simple tasks, you may not even need any instructions; examples can be enough. Of course, best practice would be to include instructions anyway. However, this example below is meant to show off the power of examples.

In [7]:
response = get_response("""

Human: 

Quote:
“When the reasoning mind is forced to confront the impossible again and again, it has no choice but to adapt.”
― N.K. Jemisin, The Fifth Season
Author: N.K. Jemisin

Quote:
“Some humans theorize that intelligent species go extinct before they can expand into outer space. If they're correct, then the hush of the night sky is the silence of the graveyard.”
― Ted Chiang, Exhalation
Author:

Assistant:""")

print(response)

The quote "Some humans theorize that intelligent species go extinct before they can expand into outer space. If they're correct, then the hush of the night sky is the silence of the graveyard." is by Ted Chiang from his short story collection Exhalation.


# Chapter 7 - END.